In [1]:
import mysql.connector
import csv
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [3]:
con = mysql.connector.connect(user='andr',
                              password='rstq!2Ro',
                              host='127.0.0.1',
                              database='cat_db',
                              auth_plugin='mysql_native_password'
                             )

In [4]:
curA = con.cursor(dictionary=True, buffered=True)
curB = con.cursor(dictionary=True, buffered=True)

In [5]:
curA.execute("SELECT id_unigram, unigram, freq_all, morph FROM unigrams;")
rows = curA.fetchall()

In [6]:
with open('morph_tags.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id_unigram', 'unigram', 'freq_all', 'morph'])
    for dictionary in rows:
        writer.writerow([dictionary['id_unigram'], dictionary['unigram'], dictionary['freq_all'], dictionary['morph']])

df = pd.read_csv('morph_tags.csv')

df.head()

,id_unigram,unigram,freq_all,morph
0,36215,NUM,94888,_
1,47683,<URL>,1307,_
2,381939,А,2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
3,381940,В,803,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
4,381941,Малько,45,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing


In [7]:
curB.execute("SELECT morph, freq_all*count AS 'par_count' FROM (SELECT morph, freq_all, COUNT(*) AS 'count' FROM unigrams GROUP BY morph, freq_all) AS tab1")

In [8]:
rowsB = curB.fetchall()

In [9]:
with open('stats.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['morph', 'count'])
    for dictionary in rowsB:
        writer.writerow([dictionary['morph'], dictionary['par_count']])

In [10]:
df_par_stats = pd.read_csv('stats.csv')

df_par_stats[:10]

,morph,count
0,_,94888
1,_,1307
2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,536
3,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,803
4,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,90
5,_,175057
6,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,107
7,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,70
8,_,137292
9,Case=Nom|Degree=Pos|Number=Plur,49


In [11]:
partial_stats = df_par_stats.to_dict('records')

In [12]:
stats = {}
for dictn in partial_stats:
    if dictn['morph'] in stats:
        stats[dictn['morph']] += dictn['count']
    else:
        stats[dictn['morph']] = dictn['count']

In [13]:
len(stats)

498

In [14]:
df_stats = pd.DataFrame(stats.items(), columns=['morph', 'count'])

In [15]:
df_stats = df_stats[df_stats.morph != '_']

In [17]:
df_stats.head()

,morph,count
1,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,9117
2,Case=Nom|Degree=Pos|Number=Plur,17522
3,Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur,10837
4,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,13939
5,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,39252


In [23]:
tot = df_stats['count'].sum(axis=0)

In [27]:
partial_freq = df_stats['count'] / tot

In [28]:
df_stats.insert(loc=len(df_stats.columns), column='partial_freq', value=partial_freq) 
df_stats.head()

,morph,count,partial_freq
1,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,9117,0.005567
2,Case=Nom|Degree=Pos|Number=Plur,17522,0.010699
3,Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur,10837,0.006617
4,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,13939,0.008511
5,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,39252,0.023968
